# California LAUS (Local Area Unemployment Statistics) Analysis

## Project Overview
This project analyzes California’s labor market using **Local Area Unemployment Statistics (LAUS)** data.  
The raw CSV files are cleaned and standardized, then used to explore unemployment trends at the **county** and **regional** levels.  

The notebook is designed to:
- Provide a **transparent, reproducible workflow** for LAUS data analysis.  
- Recreate Excel dashboards into **interactive visualizations** with Plotly.  
- Highlight key **county and regional unemployment trends** from 2015–2025.

---

## Goals
1. **Clean & Merge** raw LAUS datasets into one standardized dataset.  
2. **Map Areas → Counties → Regions** for more meaningful geographic comparisons.  
3. **Visualize Trends** with interactive charts:  
   - County-level comparisons   
   - Regional unemployment trends and comparisons  
4. **Derive Insights** to better understand California’s labor market patterns.  

---


In [11]:
import pandas as pd
import glob
import os

# Built-in mapping from CA area names to (County, Region)
AREA_TO_COUNTY_REGION = {
    "Anaheim-Santa Ana-Irvine, CA Met Div": ("Orange County", "Southern California"),
    "Oakland-Fremont-Berkeley, CA Met Div": ("Alameda County; Contra Costa County", "Bay Area"),
    "San Francisco-San Mateo-Redwood City, CA Met Div": ("San Francisco County; San Mateo County", "Bay Area"),
    "Bakersfield-Delano, CA MSA": ("Kern County", "Central Valley"),
    "El Centro, CA MSA": ("Imperial County", "Southern California"),
    "San Rafael, CA Met Div": ("Marin County", "Bay Area"),
    "Los Angeles-Long Beach-Anaheim, CA MSA": ("Los Angeles County; Orange County", "Southern California"),
    "Sacramento-Roseville-Folsom, CA MSA": ("Sacramento County; Placer County; El Dorado County; Yolo County", "Northern California"),
    "Los Angeles-Long Beach-Glendale, CA Met Div": ("Los Angeles County", "Southern California"),
    "San Diego-Carlsbad, CA MSA": ("San Diego County", "Southern California"),
    "Oxnard-Thousand Oaks-Ventura, CA MSA": ("Ventura County", "Southern California"),
    "San Diego-Chula Vista-Carlsbad, CA MSA": ("San Diego County", "Southern California"),
    "Riverside-San Bernardino-Ontario, CA MSA": ("Riverside County; San Bernardino County", "Inland Empire"),
    "San Francisco-Redwood City-South San Francisco, CA Met Div": ("San Francisco County; San Mateo County", "Bay Area"),
    "Oakland-Hayward-Berkeley, CA Met Div": ("Alameda County; Contra Costa County", "Bay Area"),
    "San Francisco-Oakland-Fremont, CA MSA": ("San Francisco County; Alameda County; Contra Costa County", "Bay Area"),
    "San Jose-Sunnyvale-Santa Clara, CA MSA": ("Santa Clara County; San Benito County", "Bay Area"),
    "Vallejo-Fairfield, CA MSA": ("Solano County", "Bay Area"),
    "Napa, CA MSA": ("Napa County", "Bay Area"),
    "Santa Rosa-Petaluma, CA MSA": ("Sonoma County", "Bay Area"),
    "Santa Cruz-Watsonville, CA MSA": ("Santa Cruz County", "Central Coast"),
    "Salinas, CA MSA": ("Monterey County", "Central Coast"),
    "San Luis Obispo-Paso Robles-Arroyo Grande, CA MSA": ("San Luis Obispo County", "Central Coast"),
    "Santa Maria-Santa Barbara, CA MSA": ("Santa Barbara County", "Central Coast"),
    "Bakersfield, CA MSA": ("Kern County", "Central Valley"),
    "Fresno, CA MSA": ("Fresno County", "Central Valley"),
    "Hanford-Corcoran, CA MSA": ("Kings County", "Central Valley"),
    "Madera, CA MSA": ("Madera County", "Central Valley"),
    "Merced, CA MSA": ("Merced County", "Central Valley"),
    "Modesto, CA MSA": ("Stanislaus County", "Central Valley"),
    "Stockton-Lodi, CA MSA": ("San Joaquin County", "Central Valley"),
    "Visalia-Porterville, CA MSA": ("Tulare County", "Central Valley"),
    "Yuba City, CA MSA": ("Sutter County; Yuba County", "Northern California"),
    "Chico, CA MSA": ("Butte County", "Northern California"),
    "Redding, CA MSA": ("Shasta County", "Northern California"),
    "Sacramento-Roseville-Arden-Arcade, CA MSA": ("Sacramento County; Placer County; El Dorado County; Yolo County", "Northern California"),
}


def load_area_mapping(optional_csv_path: str) -> dict:
    mapping = dict(AREA_TO_COUNTY_REGION)
    return mapping


def map_county_region(area_name: str, mapping: dict) -> tuple:
    if not isinstance(area_name, str):
        return ("Unknown", "Unknown")
    name = area_name.strip()
    if name in mapping:
        return mapping[name]
    # Attempt to trim known suffixes and match
    suffixes = [", CA Met Div", ", CA MSA", ", CA"]
    for suffix in suffixes:
        if name.endswith(suffix):
            trimmed = name[: -len(suffix)]
            for key in mapping.keys():
                if key.startswith(trimmed):
                    return mapping[key]
    return ("Unknown", "Unknown")


path = os.path.join("..", "data", "*.csv")

all_files = glob.glob(path)
print("Looking for files in:", os.path.abspath(os.path.join("..", "data")))
print("Files found:", all_files)

if not all_files:
    raise FileNotFoundError("No CSV files found in the data folder. Check path!")

# Load all CSVs
df_list = [pd.read_csv(file, encoding='utf-8') for file in all_files]
df = pd.concat(df_list, ignore_index=True)

df.columns = df.columns.str.strip()

# Convert Year to integer

df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
df = df[df['Year'].notna()]
df['Year'] = df['Year'].astype(int)

columns_to_keep = ["Area Name", "Year", "Month", "Labor Force", "Employment", "Unemployment", "Unemployment Rate"]
df = df[columns_to_keep]

# Assign County and Region based on Area Name
mapping_csv = 'Excel/california_laus_project/data/area_to_county_region.csv'
area_mapping = load_area_mapping(mapping_csv)
df[["County", "Region"]] = df["Area Name"].apply(lambda n: pd.Series(map_county_region(n, area_mapping)))


df = df[df["Year"] >= 2015] # Filter for last 10 years (2015-2025)

output_path = os.path.abspath(os.path.join("..", "data", "cleaned_data_notebook.csv"))
print("Saving to:", output_path)

df.to_csv(output_path, index=False)
print("Data cleaned and saved to 'cleaned_data_notebook.csv'.")

Looking for files in: c:\Users\Austin\Documents\GitHubRepo\data-analyst-portfolio\Excel\california_laus_project\data
Files found: ['..\\data\\ca-msa-sa-labor-force-2506.csv', '..\\data\\cleaned_data.csv', '..\\data\\cleaned_data_notebook.csv']
Saving to: c:\Users\Austin\Documents\GitHubRepo\data-analyst-portfolio\Excel\california_laus_project\data\cleaned_data_notebook.csv
Data cleaned and saved to 'cleaned_data_notebook.csv'.


## Exploratory Visualizations

We now create three interactive visuals that replicate and expand upon the Excel dashboard PDFs:

1. **County-level comparison line graph** – compares unemployment rates across counties.  
2. **County unemployment trends by year** – shows year-over-year trends for each county.  
3. **Regional unemployment trends** – compares broader regions of California.  

All figures are interactive:  
- Hover tooltips  
- Zoom & pan  
- Clickable legends to show/hide counties/regions  


In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

df = pd.read_csv("../data/cleaned_data_notebook.csv")
df['Unemployment Rate'] = (
    df['Unemployment Rate']
    .astype(str)
    .str.replace('%', '', regex=False)
    .str.strip()
    .replace('', np.nan)
    .astype(float)
)

county_trends = df.groupby(["Year", "County"])["Unemployment Rate"].mean().reset_index()
print(county_trends.head())

# County Unemployment Trends by Year
fig1 = px.line(
    county_trends,
    x="Year",
    y="Unemployment Rate",
    color="County",
    markers=True,
    title="County-Level Unemployment Comparison (2015–2025)",
    labels={"Unemployment Rate": "Unemployment Rate (%)"}
)
fig1.update_layout(legend_title="County", hovermode="x unified")
fig1.show()

Unemployment Rate dtype after cleaning: float64
Any NaNs in Unemployment Rate after cleaning? 0
county_trends head:
    Year                               County  Unemployment Rate
0  2015  Alameda County; Contra Costa County           4.875000
1  2015                         Butte County           7.208333
2  2015                        Fresno County          10.266667
3  2015                      Imperial County          24.750000
4  2015                          Kern County          10.225000


In [ ]:
# Regional Unemployment Trends by Year
region_trends = df.groupby(["Year", "Region"])["Unemployment Rate"].mean().reset_index()

fig3 = px.line(
    region_trends,
    x="Year",
    y="Unemployment Rate",
    color="Region",
    markers=True,
    title="Regional Unemployment Trends (2015–2025)",
    labels={"Unemployment Rate": "Unemployment Rate (%)"}
)
fig3.update_layout(
    legend_title="Region",
    hovermode="x unified",
    xaxis=dict(dtick=1)
)
fig3.show()

## Insights

From the visualizations we observe:  
- **Unemployment spikes in 2020** across nearly all counties and regions (COVID-19).  
- **Central Valley counties** consistently show higher unemployment and greater volatility than coastal regions.  
- **Bay Area and Northern California** tend to have lower unemployment rates compared to Southern California.  
- **Regional disparities persist**, with Inland Empire and Central Valley trailing behind other regions.  

---

## Conclusion

This notebook demonstrates how California LAUS data can be:  
- **Cleaned and standardized** for long-term analysis (2015–2025).  
- **Mapped from raw area names** to counties and broader regions.  
- **Explored interactively** with dynamic visualizations that replicate and enhance the Excel dashboard.

### Next Steps
- Add **forecasting models** (ARIMA, Prophet) to predict unemployment trends.  
- Combine LAUS with **Census demographic data** to analyze socioeconomic factors.  
- Build a full **interactive dashboard app** (using Plotly Dash or Streamlit) for stakeholders.  
